# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the cities.csv to create dataframe
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)

humidity = cities["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) &(cities["Wind Speed"] < 10) & (cities["Cloudiness"]==0),:]
ideal_cities = ideal_cities.dropna(how='any')
ideal_cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# loop through df

for index, row in ideal_cities.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # set up a parameters dictionary
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    hotel_data = requests.get(base_url, params=params).json()

    print(json.dumps(hotel_data, indent=4, sort_keys=True))
    
    try:
        ideal_cities.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "next_page_token": "Aap_uECbBG91jBTo792gryeAr5mH63IX8Fl1YGtKwCR_ycp_J-9eXndam6YM9QoY2CKdDJn3X_ELqISaDpP8FIoCasrS0IET5aQKAov5pIZ7pZzBlHBIu_9LkaoGuu9eo1K_3dUck7x9mtf6-ltOIdt_ThP2gV_k_rOdTtAl6L6N3IQ6jlOjKRcB2Gnre-CYAHpi4MWg1UT6DMkPDZjfX1i9XpL2DMMAk-UhU5ZFP77lqXsWkRjXAuHNeI1Yew8OLOjErdwl-2YHs1PoDc0CrA4VGWSE1IkgboGyvrRoO7vYSfbpX1c1t3gqKqQ6X54BiXvrGvR_Evs9otJerjNuVnAW1D-EOkdTLPCLNWrmYtEKMRAvR0oMthV4Z22OVzkSGjCTzoYfQVxgtATUWBkvZr_ouqKuzm1PkpwNBUpXl7mU5sNCXpz1fX9gyqbBQkK7OrA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6456114,
                    "lng": -46.0097467
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64429397010727,
                        "lng": -46.00845397010728
                    },
                    "southwest": {
                        "lat": -20.64699

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -30.1778807,
                    "lng": -50.2035006
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.17650502010729,
                        "lng": -50.20220167010728
                    },
                    "southwest": {
                        "lat": -30.17920467989273,
                        "lng": -50.20490132989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Castelo",
            "photos": [
                {
                    "height": 3456

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 15.346751,
                    "lng": 74.5356042
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.352004,
                        "lng": 74.53854345
                    },
                    "southwest": {
                        "lat": 15.34500000000001,
                        "lng": 74.53462445
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Dandeli Aura Jungle Stay",
            "photos": [
                {
                    "height": 720,
               

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.9075026,
                    "lng": 96.4755053
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.90877412989272,
                        "lng": 96.47683007989272
                    },
                    "southwest": {
                        "lat": 22.90607447010728,
                        "lng": 96.47413042010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "King Bridge",
            "opening_hours": {
                "open_now": true
            },
     

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8799738,
                    "lng": 59.21613479999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88132187989272,
                        "lng": 59.21742852989271
                    },
                    "southwest": {
                        "lat": 32.87862222010727,
                        "lng": 59.21472887010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "\u0645\u0647\u0645\u0627\u0646 \u067e\u0630\u06cc\u0631 \u0646\u0648\u06cc\u062f",
        

In [8]:
hotel_df = ideal_cities

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Kimony Resort Hotel
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Aura Jungle Stay
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,King Bridge
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,مهمان پذیر نوید


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))